<a href="https://colab.research.google.com/github/hamidzangiabadi/sailfish-optimization-algorithm/blob/main/Method.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install ucimlrepo
# !pip install mealpy
# !pip install scikit-learn
!pip install uci-dataset

# import datasets

In [1]:
from ucimlrepo import fetch_ucirepo

# fetch dataset
# wine = fetch_ucirepo(id=109)
# breast_cancer = fetch_ucirepo(id=14)
zoo = fetch_ucirepo(id=111)
# automobile = fetch_ucirepo(id=10)
# print("imported")
# lymphography = fetch_ucirepo(id=63)
# print("imported")
# student_performance = fetch_ucirepo(id=320) 
# ionosphere = fetch_ucirepo(id=52)
# credit_approval = fetch_ucirepo(id=27) 
# hepatitis = fetch_ucirepo(id=46) 

print("imported")


imported


In [2]:
import pandas as pd


Datasets = [
#             { "name" : "Wine" , "dataset" : wine },
#              { "name" : "Breast Cancer" , "dataset" : breast_cancer },
             { "name" : "Zoo" , "dataset" : zoo },
#              { "name" : "Automobile" , "dataset" : automobile },
#              { "name" : "Lymphography" , "dataset" : lymphography },
#              { "name" : "Student Performance" , "dataset" : student_performance },
#              { "name" : "Ionosphere" , "dataset" : ionosphere },
#              { "name" : "Credit Approval" , "dataset" : credit_approval },
#              { "name" : "Hepatitis" , "dataset" : hepatitis },
           ]


# def save_dataset(dataset_info):
#     name = dataset_info["name"]
#     dataset = dataset_info["dataset"]
    
#     # Access the features and targets
#     X = dataset.data
#     # Convert features and target to DataFrame
#     df_X = pd.DataFrame(X)
#     df_y = pd.DataFrame(y)
    
#     print(df_y)
    
#     # Combine features and target into a single DataFrame
#     df = pd.concat([df_X, df_y], axis=1)
#     print(df)

#     # Save to CSV
#     file_name = f"{name.replace(' ', '_').lower()}.csv"
#     df.to_csv(file_name, index=False)
#     print(f"Saved {name} dataset to {file_name}")

# # Save all datasets
# for dataset_info in Datasets:
#     save_dataset(dataset_info)



In [3]:
# with open('dotdict.pkl', 'rb') as f:
#     loaded_dotdict = pickle.load(f)
#     print(loaded_dotdict)


## modify datasets and required information





Preprocessing to convert non int and float columns using label encoder

In [4]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd



for x in Datasets:

  label_encoder = LabelEncoder()
  df = x["dataset"].data.features
  object_columns = df.select_dtypes(include=['object']).columns
  # Apply LabelEncoder to each object column
  for column in object_columns:
      df[column + '_encoded'] = label_encoder.fit_transform(df[column])
  df = df.drop(columns=object_columns)

  x["dataset"].data.features = df

  targetdf = x["dataset"].data.targets
  object_columns = targetdf.select_dtypes(include=['object']).columns
  # Apply LabelEncoder to each object column
  for column in object_columns:
      targetdf[column + '_encoded'] = label_encoder.fit_transform(targetdf[column])
  targetdf = targetdf.drop(columns=object_columns)

  x["dataset"].data.targets = targetdf


## functions

In [13]:
import csv

def binary_conversion(X, dim):
    Xbin = np.zeros(dim)
    for d in range(dim):
        if X[d] > 0.5:
            Xbin[d] = 1
        else:
            Xbin[d] = 0

    return Xbin


def save_results(g_best, method_name, num_features,datasetName):
    
    selectedFeaturesSubset = binary_conversion(g_best.solution,num_features)
    featuresCount = int(sum(selectedFeaturesSubset))
    bestFitness = g_best.target.fitness
    
    print(f"Solution: {str(selectedFeaturesSubset)}, Fitness: {g_best.target.fitness}")
    print(f"No of features: {str(featuresCount)}")
    print(f"Reached accuracy: {str(checkAccuracy(selectedFeaturesSubset))}");
    
  
    res = {
        "Method": method_name,
        "NumberOfFeatures": str(featuresCount),
        "Accuracy": checkAccuracy(selectedFeaturesSubset),
        "Fitness": bestFitness
    }
    # Save to CSV or any other preferred format
    # Example: Using pandas to save as CSV
    import pandas as pd
    df = pd.DataFrame([res])
    df.to_csv(f"{datasetName}_results.csv", mode='a', index=False, header=not pd.io.common.file_exists(f"{datasetName}_results.csv"))
    

## method

In [ ]:
import numpy as np
from mealpy import FloatVar, GA,SFO,WOA,EHO,GWO,DO,MVO,MFO
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

X_train, X_test, y_train, y_test = "", "" ,"" ,""



def checkAccuracy(features):
    max_feat = len(features)
    num_feat = np.sum(binary_conversion(features,num_features) == 1)
    if num_feat == 0:
        return 0

    targets = np.array(y).ravel()
    selected_features = binary_conversion(features,num_features);
    f =  X.iloc[:, selected_features == 1]
    X_train, X_test, y_train, y_test = train_test_split(f, targets, test_size=0.3)
    svm = SVC(kernel='linear')
    svm.fit(X_train, y_train.ravel())
    # Predict on test set
    y_pred = svm.predict(X_test)
    Ytest = np.array(y_test)
    num_valid = np.size(X_test, 0)
    yvalid  = Ytest.reshape(num_valid)
    accuracy = accuracy_score(y_test, y_pred)
    print(accuracy)
    return accuracy

def objective_function(features, X_train, X_test, y_train, y_test):
    max_feat = len(features)
    num_feat = np.sum(binary_conversion(features,num_features) == 1)
    
    # If no features or all features are selected, return a poor score
    if num_feat == 0 or num_feat == max_feat:
        return 0  # Adjusted to return a low score, not 1


    alpha = 0.6
    beta = 0.4

    targets = np.array(y)
    selected_features = binary_conversion(features, X_train.shape[1])
    f_train = X_train.iloc[:, selected_features == 1]
    f_test = X_test.iloc[:, selected_features == 1]
    
    svm = SVC(kernel='linear')
    svm.fit(f_train, np.array(y_train).ravel())
    y_pred = svm.predict(f_test)

    accuracy = accuracy_score(y_test, y_pred)

    cost = alpha * accuracy - beta * (num_feat / max_feat)
    return cost



for dataset in Datasets:
    X = dataset["dataset"].data.features
    y = dataset["dataset"].data.targets

    num_features = X.shape[1]
    search_space = np.array([0, 1] * num_features)

    lb = np.zeros(num_features)
    ub = np.ones(num_features)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
    
   
    
    problem_dict = {
    "bounds": FloatVar(lb=lb, ub=ub, ),
    "obj_func": lambda features: objective_function(features, X_train, X_test, y_train, y_test),
    "minmax": "max",
    }
    
    for i in range(10):
    
        SFOModel = SFO.OriginalSFO(epoch=30,pop_size=50, pp = 0.1, AP =4 ,epsilon = 0.001)
        g_best = SFOModel.solve(problem_dict)
        save_results(g_best,"SFO", num_features, dataset["name"])

        GAmodel = GA.BaseGA(epoch=30,pop_size=50,pc=0.7, pm=0.05)
        g_best = GAmodel.solve(problem_dict)
        save_results(g_best,"GA", num_features, dataset["name"])

        EHOModel = EHO.OriginalEHO(epoch=30,pop_size=50, alpha = 0.5, beta = 0.5, n_clans = 2)
        g_best = EHOModel.solve(problem_dict)
        save_results(g_best,"EHO", num_features, dataset["name"])

        GWOModel = GWO.OriginalGWO(epoch=30,pop_size=50)
        g_best = GWOModel.solve(problem_dict)
        save_results(g_best,"GWO", num_features, dataset["name"])

        WOAModel = WOA.OriginalWOA(epoch=30,pop_size=50)
        g_best = WOAModel.solve(problem_dict)
        save_results(g_best,"WOA", num_features, dataset["name"])

        MFOModel = MFO.OriginalMFO(epoch=30,pop_size=50)
        g_best = MFOModel.solve(problem_dict)
        save_results(g_best,"MFO", num_features, dataset["name"])


2024/05/18 09:59:12 PM, INFO, mealpy.swarm_based.SFO.OriginalSFO: Solving single objective optimization problem.
2024/05/18 09:59:15 PM, INFO, mealpy.swarm_based.SFO.OriginalSFO: >>>Problem: P, Epoch: 1, Current best: 0.4612903225806452, Global best: 0.4612903225806452, Runtime: 0.49949 seconds
2024/05/18 09:59:16 PM, INFO, mealpy.swarm_based.SFO.OriginalSFO: >>>Problem: P, Epoch: 2, Current best: 0.4612903225806452, Global best: 0.4612903225806452, Runtime: 0.30172 seconds
2024/05/18 09:59:16 PM, INFO, mealpy.swarm_based.SFO.OriginalSFO: >>>Problem: P, Epoch: 3, Current best: 0.46693548387096767, Global best: 0.46693548387096767, Runtime: 0.34479 seconds
2024/05/18 09:59:16 PM, INFO, mealpy.swarm_based.SFO.OriginalSFO: >>>Problem: P, Epoch: 4, Current best: 0.46693548387096767, Global best: 0.46693548387096767, Runtime: 0.50100 seconds
2024/05/18 09:59:17 PM, INFO, mealpy.swarm_based.SFO.OriginalSFO: >>>Problem: P, Epoch: 5, Current best: 0.46693548387096767, Global best: 0.4669354838

Solution: [0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0.], Fitness: 0.46693548387096767
No of features: 3
0.8709677419354839
Reached accuracy: 0.8709677419354839
0.9032258064516129


2024/05/18 09:59:20 PM, INFO, mealpy.evolutionary_based.GA.BaseGA: >>>Problem: P, Epoch: 1, Current best: 0.42258064516129035, Global best: 0.42258064516129035, Runtime: 0.29721 seconds
2024/05/18 09:59:20 PM, INFO, mealpy.evolutionary_based.GA.BaseGA: >>>Problem: P, Epoch: 2, Current best: 0.46693548387096767, Global best: 0.46693548387096767, Runtime: 0.30035 seconds
2024/05/18 09:59:21 PM, INFO, mealpy.evolutionary_based.GA.BaseGA: >>>Problem: P, Epoch: 3, Current best: 0.46693548387096767, Global best: 0.46693548387096767, Runtime: 0.29758 seconds
2024/05/18 09:59:21 PM, INFO, mealpy.evolutionary_based.GA.BaseGA: >>>Problem: P, Epoch: 4, Current best: 0.46693548387096767, Global best: 0.46693548387096767, Runtime: 0.29893 seconds
2024/05/18 09:59:21 PM, INFO, mealpy.evolutionary_based.GA.BaseGA: >>>Problem: P, Epoch: 5, Current best: 0.46693548387096767, Global best: 0.46693548387096767, Runtime: 0.30473 seconds
2024/05/18 09:59:21 PM, INFO, mealpy.evolutionary_based.GA.BaseGA: >>>

Solution: [0. 1. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0.], Fitness: 0.46693548387096767
No of features: 3
0.7741935483870968
Reached accuracy: 0.7741935483870968
0.9032258064516129


2024/05/18 09:59:30 PM, INFO, mealpy.swarm_based.EHO.OriginalEHO: >>>Problem: P, Epoch: 1, Current best: 0.4419354838709677, Global best: 0.4419354838709677, Runtime: 0.40330 seconds
2024/05/18 09:59:30 PM, INFO, mealpy.swarm_based.EHO.OriginalEHO: >>>Problem: P, Epoch: 2, Current best: 0.4419354838709677, Global best: 0.4419354838709677, Runtime: 0.34807 seconds
2024/05/18 09:59:30 PM, INFO, mealpy.swarm_based.EHO.OriginalEHO: >>>Problem: P, Epoch: 3, Current best: 0.46693548387096767, Global best: 0.46693548387096767, Runtime: 0.31965 seconds
2024/05/18 09:59:31 PM, INFO, mealpy.swarm_based.EHO.OriginalEHO: >>>Problem: P, Epoch: 4, Current best: 0.46693548387096767, Global best: 0.46693548387096767, Runtime: 0.29902 seconds
2024/05/18 09:59:31 PM, INFO, mealpy.swarm_based.EHO.OriginalEHO: >>>Problem: P, Epoch: 5, Current best: 0.46693548387096767, Global best: 0.46693548387096767, Runtime: 0.29925 seconds
2024/05/18 09:59:31 PM, INFO, mealpy.swarm_based.EHO.OriginalEHO: >>>Problem: P

...

Solution: [0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0.], Fitness: 0.46693548387096767
No of features: 3
0.967741935483871
Reached accuracy: 0.967741935483871
0.8064516129032258


2024/05/18 10:02:13 PM, INFO, mealpy.swarm_based.EHO.OriginalEHO: >>>Problem: P, Epoch: 1, Current best: 0.3975806451612903, Global best: 0.3975806451612903, Runtime: 0.34875 seconds
2024/05/18 10:02:13 PM, INFO, mealpy.swarm_based.EHO.OriginalEHO: >>>Problem: P, Epoch: 2, Current best: 0.4419354838709677, Global best: 0.4419354838709677, Runtime: 0.30914 seconds
2024/05/18 10:02:14 PM, INFO, mealpy.swarm_based.EHO.OriginalEHO: >>>Problem: P, Epoch: 3, Current best: 0.4419354838709677, Global best: 0.4419354838709677, Runtime: 0.32704 seconds
2024/05/18 10:02:14 PM, INFO, mealpy.swarm_based.EHO.OriginalEHO: >>>Problem: P, Epoch: 4, Current best: 0.4419354838709677, Global best: 0.4419354838709677, Runtime: 0.39235 seconds
2024/05/18 10:02:14 PM, INFO, mealpy.swarm_based.EHO.OriginalEHO: >>>Problem: P, Epoch: 5, Current best: 0.4419354838709677, Global best: 0.4419354838709677, Runtime: 0.31813 seconds
2024/05/18 10:02:15 PM, INFO, mealpy.swarm_based.EHO.OriginalEHO: >>>Problem: P, Epoc

Solution: [0. 0. 1. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0.], Fitness: 0.4419354838709677
No of features: 4
0.7096774193548387
Reached accuracy: 0.7096774193548387
0.8387096774193549


2024/05/18 10:02:23 PM, INFO, mealpy.swarm_based.GWO.OriginalGWO: >>>Problem: P, Epoch: 1, Current best: 0.43629032258064515, Global best: 0.43629032258064515, Runtime: 0.29131 seconds
2024/05/18 10:02:23 PM, INFO, mealpy.swarm_based.GWO.OriginalGWO: >>>Problem: P, Epoch: 2, Current best: 0.46693548387096767, Global best: 0.46693548387096767, Runtime: 0.28885 seconds
2024/05/18 10:02:23 PM, INFO, mealpy.swarm_based.GWO.OriginalGWO: >>>Problem: P, Epoch: 3, Current best: 0.46693548387096767, Global best: 0.46693548387096767, Runtime: 0.28806 seconds
2024/05/18 10:02:24 PM, INFO, mealpy.swarm_based.GWO.OriginalGWO: >>>Problem: P, Epoch: 4, Current best: 0.46693548387096767, Global best: 0.46693548387096767, Runtime: 0.28973 seconds
2024/05/18 10:02:24 PM, INFO, mealpy.swarm_based.GWO.OriginalGWO: >>>Problem: P, Epoch: 5, Current best: 0.46693548387096767, Global best: 0.46693548387096767, Runtime: 0.29170 seconds
2024/05/18 10:02:24 PM, INFO, mealpy.swarm_based.GWO.OriginalGWO: >>>Proble

Solution: [0. 0. 0. 1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0.], Fitness: 0.46693548387096767
No of features: 3
0.8387096774193549
Reached accuracy: 0.8387096774193549
0.9032258064516129


# create plots

In [ ]:

import numpy as np
import matplotlib.pyplot as plt



# set width of bar
barWidth = 0.25
fig = plt.subplots(figsize =(12, 8))


SFO = [];
GA = [];

DatasetsName = []


for x in Datasets:
  SFO.append(float(x["Accuracy_SFO"]) * 100)
  GA.append(float(x["Accuracy_GA"]) * 100)
  DatasetsName.append(x["name"])


# Set position of bar on X axis
br1 = np.arange(len(SFO))
br2 = [x + barWidth for x in br1]
br3 = [x + barWidth for x in br2]

# Make the plot
plt.bar(br1, SFO, color ='r', width = barWidth,
        edgecolor ='grey', label ='SFO')
plt.bar(br2, GA, color ='g', width = barWidth,
        edgecolor ='grey', label ='GA')

# Adding Xticks
plt.xlabel('Branch', fontweight ='bold', fontsize = 15)
plt.ylabel('Students passed', fontweight ='bold', fontsize = 15)
plt.xticks([r + barWidth for r in range(len(SFO))],
       DatasetsName)

plt.legend()
plt.show()